In [1]:
%pip install tabgan=='1.1.0'
%pip install Faker
%pip install catboost

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from faker import Faker
import random

random.seed(10)
DATA_SIZE = 5000

In [3]:
fake = Faker()
Faker.seed(42)
names = []
for i in range(0,DATA_SIZE):
    names.append(fake.name())

In [4]:
Faker.seed(42)
state = []
for i in range(0,DATA_SIZE):
    state.append(fake.state())

In [5]:
specialty = []
for i in range(0, DATA_SIZE):
    specialty.append("Emergency Medicine")

In [6]:
from faker.providers import DynamicProvider
health_insurance_provider = DynamicProvider(
     provider_name="health_insurance",
     elements=["UnitedHealth Group", "Anthem", "Aetna", "Cigna", "Humana", "Medicare"],
)
Faker.seed(42)

fake.add_provider(health_insurance_provider)

insurance = []
for i in range(0, DATA_SIZE):
    insurance.append(fake.health_insurance())

In [7]:
Faker.seed(42)
sex = []
for i in range(0, DATA_SIZE):
    sex.append(fake.profile()['sex'])

In [8]:
import numpy as np

np.random.seed(42)
mu, sigma = 50, 20
age = np.random.normal(mu, sigma, DATA_SIZE)
age_int = np.round(age)

In [9]:
np.random.seed(42)
mu, sigma = 180, 70
weight = np.round(np.random.normal(mu, sigma, DATA_SIZE), 2)

In [10]:
np.random.seed(42)
mu, sigma = 68, 8
height = np.round(np.random.normal(mu, sigma, DATA_SIZE), 2)

In [11]:
smoker = []
for i in range(0, DATA_SIZE):
    smoker.append(np.random.randint(0,2))

smoker_category = ['Yes' if x == 1 else 'No' for x in smoker]

In [12]:
np.random.seed(42)
legnth_of_stay = []
for i in range(0, DATA_SIZE):
    legnth_of_stay.append(np.random.randint(1,10))

In [13]:
import pandas as pd 
df = pd.DataFrame({'height':height, 'weight':weight, 
                'age':age_int, 'sex':sex, 'insurance':insurance, 
                'specialty':specialty, 'name':names, 'state':state,
                'length_of_stay':legnth_of_stay, 'smoker':smoker_category,
                })

In [14]:
df['bmi'] = np.round((df['weight']/(df['height']**2))*703, 3)

In [ ]:
df['readmission'] = [np.random.randint(0,2) for x in range(0, DATA_SIZE)]
df.drop_duplicates('name', inplace=True)
np.random.seed(42)

df_sample1 = df.sample(frac=0.2, replace=True, random_state=1)

df_sample2 = df[~df['name'].isin(list(set(df_sample1['name'])))]

df_sample2['readmission'] = 0 
df_sample2.loc[(df_sample2.bmi >=30) & (df_sample2.smoker == 'Yes') & (df_sample2.insurance == 'Medicare') & (df.length_of_stay >= 5), 'readmission'] = 1

df = df_sample2.append(df_sample1)

/tmp/ipykernel_482/3297978708.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample2['readmission'] = 0
/shared-libs/python3.8/py/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
from collections import Counter
Counter(df['readmission'])

Counter({0: 4405, 1: 517})

In [ ]:
df['insurance'] = df['insurance'].astype('category')
df['insurance'] = df['insurance'].cat.codes

df['sex'] = df['sex'].astype('category')
df['sex'] = df['sex'].cat.codes

df['smoker'] = df['smoker'].astype('category')
df['smoker'] = df['smoker'].cat.codes

df['state'] = df['state'].astype('category')
df['state'] = df['state'].cat.codes

In [ ]:
X = df[['insurance', 'sex', 'smoker', 'state', 'height', 'weight', 'bmi', 'length_of_stay']]
y = df['readmission']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cats = ['insurance', 'sex', 'smoker', 'state']
model1 = CatBoostClassifier(cat_features= cats, iterations=10)
model1.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.4660763	total: 48.2ms	remaining: 434ms
1:	learn: 0.3911385	total: 49.5ms	remaining: 198ms
2:	learn: 0.3574981	total: 50.5ms	remaining: 118ms
3:	learn: 0.3447903	total: 51.3ms	remaining: 76.9ms
4:	learn: 0.3374272	total: 52.3ms	remaining: 52.3ms
5:	learn: 0.3343143	total: 53.4ms	remaining: 35.6ms
6:	learn: 0.3336231	total: 54.2ms	remaining: 23.2ms
7:	learn: 0.3302381	total: 55.2ms	remaining: 13.8ms
8:	learn: 0.3272703	total: 56.2ms	remaining: 6.24ms
9:	learn: 0.3271818	total: 57ms	remaining: 0us


In [ ]:
y_pred = model1.predict(X_test)
print(Counter(y_pred))
print(Counter(y_test))
from sklearn.metrics import precision_score
print("precision_score: ", precision_score(y_test, y_pred))

Counter({0: 1477})
Counter({0: 1331, 1: 146})
precision_score:  0.0
/shared-libs/python3.8/py/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from tabgan.sampler import GANGenerator
cols = ['insurance', 'sex', 'smoker', 'state', 'height', 'weight', 'bmi', 'length_of_stay', 'readmission']

X = df[cols[:-1]]
y = pd.DataFrame(list(df['readmission']), columns=['readmission'])
X.reset_index(inplace=True, drop=True)
y.reset_index(inplace=True, drop=True)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)

new_train2, new_target2 = GANGenerator(cat_cols = cols, epochs=2, is_post_process=False).generate_data_pipe(X_train, y_train, X_test,use_adversarial=False, only_generated_data=False)


/shared-libs/python3.8/py/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Training CTGAN, epochs:: 100%|██████████| 2/2 [00:15<00:00,  7.81s/it]


In [ ]:
print(Counter(new_target2))

Counter({0: 8102, 1: 2921})


In [ ]:
new_train2.head()


,insurance,sex,smoker,state,height,weight,bmi,length_of_stay
0,2,0,0,13,54.62,62.90,14.822,5
1,1,1,1,34,76.08,250.66,30.444,4
2,5,0,0,0,67.56,176.11,27.124,6
3,3,1,1,23,63.21,138.11,24.300,6
4,2,1,1,8,68.85,187.40,27.792,1


In [ ]:
cats = ['insurance', 'sex', 'smoker', 'state']
model2 = CatBoostClassifier(cat_features= cats, iterations=10)
model2.fit(new_train2, new_target2)

Learning rate set to 0.5
0:	learn: 0.6802753	total: 1.61ms	remaining: 14.5ms
1:	learn: 0.6708204	total: 3.13ms	remaining: 12.5ms
2:	learn: 0.6643163	total: 4.39ms	remaining: 10.3ms
3:	learn: 0.6596202	total: 5.67ms	remaining: 8.51ms
4:	learn: 0.6588420	total: 6.93ms	remaining: 6.93ms
5:	learn: 0.6580290	total: 8.15ms	remaining: 5.43ms
6:	learn: 0.6572360	total: 9.41ms	remaining: 4.03ms
7:	learn: 0.6555131	total: 10.6ms	remaining: 2.64ms
8:	learn: 0.6538899	total: 11.8ms	remaining: 1.31ms
9:	learn: 0.6511306	total: 13ms	remaining: 0us


In [ ]:
y_pred2 = model2.predict(X_test)
print(Counter(y_pred2))
print(Counter(y_test['readmission']))
from sklearn.metrics import precision_score
print("precision_score: ", precision_score(y_test, y_pred2))

Counter({0: 1471, 1: 6})
Counter({0: 1331, 1: 146})
precision_score:  0.16666666666666666


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1adae434-40f7-494d-b18c-c2d2f104fb0b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>